In [1]:
import io
import pathlib
import subprocess
import multiprocess

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageFile
from tqdm import tqdm

from core.io_util import FileStream
from core.wpg import WPGFile

In [ ]:
class OOPImageFile(ImageFile.ImageFile):
    format = "OOP"

    def _open(self):
        self.fp: io.FileIO
        reader = FileStream(self.fp)
        oop_header = reader.read(128)
        file_size = reader.read_int(4)

        unk1 = reader.read_byte_array(4)
        # print(unk1)

        count = reader.read_int(4)

        offsets = []

        for _ in range(count):
            offset = reader.read_int(4)
            offsets.append(offset)
            print(f"offset = {offset:X}")

        file_size_dupe = reader.read_int(4)

        reader.seek(offsets[0])

        wpg_header = reader.read_byte_array(16)
        wpg_temp = reader.read_string()

        wpg_magic1 = reader.read_byte_array(20)

        wpg_id = reader.read_string()

        wpg_magic2 = reader.read_byte_array(4)

        wpg_n_images = reader.read_int(4)

        wpg_magicff = reader.read_byte_array(65)
        wpg_unk1 = reader.read_int()

        self._size = (512, 256)


Image.register_open(OOPImageFile.format, OOPImageFile, lambda prefix: prefix[:3] == b"OOP")

Image.register_extensions(
    OOPImageFile.format,
    [".oop", ".1E3EE6FB"]
)

# start = 0
# pbar = tqdm(range(1000000))
# for i in pbar:
#     reader.seek(start)
#     reader.read(i)
#     rem = reader.read_remaining_bytes()
#     if len(rem) <= 0:
#         break
#     pbar.set_postfix_str(len(rem))
#     im_data = io.BytesIO()
#     try:
#         im = Image.open(im_data, formats=None)
#         print(im.width, im.height)
#     except Image.UnidentifiedImageError:
#         continue


# shape = (256, 128)
# # bytes_per_pixel = 1
# # next_read = shape[0]*shape[1]*bytes_per_pixel
# # print(f'tell={reader.tell()}, total={reader.total_bytes()}, want to read {next_read}')
# # rem_data = reader.read(next_read)
# rem_data = reader.read_remaining_bytes()
# # for mode in Image.MODES:
# #     try:
# #         im = Image.frombytes(mode, (128, 128), rem_data)
# #         plt.figure(figsize=(10, 10))
# #         plt.suptitle(mode)
# #         plt.imshow(im)
# #         plt.tight_layout()
# #     except Exception as ex:
# #         print(f'{mode} runs with exception', ex)

# # im = Image.frombytes("P", (512, 256), rem_data, "raw", "P", 0, -1)
# im = Image.frombytes("RGBA", shape, rem_data)

# # rem_data = reader.read(next_read)
# # im2 = Image.frombytes("L", shape, rem_data)

# arr = np.array(im)
# print(arr.min(), arr.max(), arr.mean(), arr.dtype, arr.shape)
# # im = Image.frombytes("F", (256, 256), rem_data, "bit", 24, 0, 0, 0, -1)
# plt.figure(figsize=(10, 10))
# plt.imshow(im, cmap='gray')
# plt.tight_layout()

# # plt.figure(figsize=(10, 10))
# # plt.imshow(im2, cmap='gray')

# print(f'count={count}, n_images={wpg_n_images}, tell=0x{reader.tell():X}, total={reader.total_bytes()}, rem={len(rem_data)}')

In [240]:
# Extract all OPK files
storage_path = pathlib.Path('textures')

opk_path = pathlib.Path(r'C:\Program Files (x86)\Steam\steamapps\common\Mega Man X Legacy Collection 2\nativeDX10\X8\romPC\data\opk')
for fpath in tqdm(list(opk_path.glob('**/*arc'))):
    output_fpath = opk_path/fpath.parent.stem/fpath.stem/'X8'/'data'/'opk'/fpath.parent.stem/f'{fpath.stem}.1E3EE6FB'
    # subprocess.call(['ARCtool.exe', '-x', '-pc', '-silent', str(output_fpath)], creationflags=subprocess.CREATE_NO_WINDOW)
    assert output_fpath.exists()

    output_fpath.rename(storage_path / output_fpath.name)

100%|██████████| 178/178 [00:00<00:00, 1508.13it/s]


In [333]:
storage_path = pathlib.Path('textures')
for fpath in tqdm(list(storage_path.glob('*.1E3EE6FB'))):
    try:
        with open(fpath, 'rb') as file:
            reader = FileStream(file)

            _ = reader.read_string(8)
            _ = reader.read_int(4)
            _ = reader.read_byte_array(4)
            count = reader.read_int(4)

            offsets = []
            for _ in range(count):
                offset = reader.read_int(4)
                offsets.append(offset)

            _ = reader.read_int(4)
            reader.seek(offsets[0])

            _ = reader.read_byte_array(16)
            _ = reader.read_string()
            _ = reader.read_byte_array(20)
            _ = reader.read_string()
            _ = reader.read_byte_array(4)
            wpg_n_images = reader.read_int()

            if count == 1 and wpg_n_images == 1:
                print('1/1', fpath)
            elif count == 1 and wpg_n_images == 2:
                print('1/2', fpath)
            elif count < 2 and wpg_n_images < 10:
                print(f'{count}/{wpg_n_images} {fpath}')
    except PermissionError:
        continue

100%|██████████| 182/182 [00:00<00:00, 10703.06it/s]

1/1 textures\2D_LOAD_ELEVATOR.1E3EE6FB
1/1 textures\2D_LOAD_ELEVATOR_copy.1E3EE6FB
1/1 textures\2D_LOAD_ONPARE.1E3EE6FB
1/1 textures\2D_LOAD_OP.1E3EE6FB
1/1 textures\2D_LOAD_SIGMA.1E3EE6FB
1/3 textures\coming.1E3EE6FB
1/1 textures\debug.1E3EE6FB
1/7 textures\font - Copy.1E3EE6FB
1/7 textures\font.1E3EE6FB
1/1 textures\HB_001.1E3EE6FB
1/1 textures\HB_002.1E3EE6FB
1/1 textures\HB_002_copy.1E3EE6FB
1/3 textures\HB_006.1E3EE6FB
1/1 textures\HB_013.1E3EE6FB
1/2 textures\intro.1E3EE6FB
1/1 textures\load.1E3EE6FB
1/5 textures\title.1E3EE6FB
1/5 textures\wep_get.1E3EE6FB


In [444]:
DEBUG = False

# for op_file in [
#                 pathlib.Path('textures/2D_LOAD_ELEVATOR.1E3EE6FB'), 
#                 pathlib.Path('textures/2D_LOAD_ONPARE.1E3EE6FB'), 
#                 pathlib.Path('textures/2D_LOAD_SIGMA.1E3EE6FB'), 
#                 pathlib.Path('textures/2D_LOAD_OP.1E3EE6FB'), 
#                 pathlib.Path('textures/intro.1E3EE6FB'), 
#                 pathlib.Path('textures/title.1E3EE6FB')]:

all_remains = []
failed_files = []
ran_out_of_bytes = []

def printd(st, *args, **kwargs):
    if DEBUG:
        print(st, *args, **kwargs)

if DEBUG:
    # l = [pathlib.Path('textures/2D_LOAD_ELEVATOR - Copy.1E3EE6FB')]
    # l = [pathlib.Path('textures/cockpit.1E3EE6FB')]
    l = [pathlib.Path('textures/2D_LOAD_SIGMA.1E3EE6FB')]
else:
    l = tqdm(list(pathlib.Path('textures').glob('*.1E3EE6FB')))

for op_file in l:
    try:
        file = open(op_file, 'rb')
    except PermissionError:
        file.close()
        printd('Cannot open due to permission')
        continue
    
    try:
        reader = FileStream(file)

        oop_header = reader.read_string(8)
        file_size = reader.read_int(4)
        unk1 = reader.read_byte_array(4)

        count = reader.read_int(4)
        printd(f'*********There are {count} files in {op_file} **********')

        offsets = []

        for _ in range(count):
            offset = reader.read_int(4)
            offsets.append(offset)
            # print(f"offset = {offset:X}")

        file_size_dupe = reader.read_int(4)

        all_n_images = {}
        for file_idx, offset in enumerate(offsets):
            reader.seek(offset)

            all_extra_ints = {}
            all_shapes = {}
            all_bpps = {}
            all_alt_w = {}
            all_alt_h = {}
            all_wpg_unk3 = {}
            
            wpg_header = reader.read_byte_array(16)
            wpg_temp = reader.read_string()

            wpg_magic1 = reader.read_byte_array(20)

            wpg_id = reader.read_string()

            wpg_magic2 = reader.read_byte_array(4)

            wpg_n_images = reader.read_int(4)
            all_n_images[offset] = wpg_n_images
            printd(f'>>> New offset {offset} with {wpg_n_images} at {reader.tell()}')

            wpg_unk1 = reader.read_byte_array(84)

            filler = 0
            for i in range(wpg_n_images):
                width = reader.read_int()
                height = reader.read_int()

                alt_w = reader.read_int()
                if alt_w == 0 or alt_w % 2 != 0 or alt_w > 512:
                    reader.seek(reader.tell()-2)
                    alt_w = reader.read_int(1)
                    reader.read_byte()
                alt_h = reader.read_int()
                if alt_h == 0 or alt_h % 2 != 0:
                    reader.seek(reader.tell()-2)
                    alt_h = reader.read_int(1)
                    reader.read_byte()

                wpg_unk2 = reader.read_int_array(4)

                bpp = reader.read_int()
                bpp_dupe = reader.read_int()

                if bpp == 8:
                    s1 = min(alt_w, width) if alt_w != 0 else width
                    s2 = min(alt_h, height) if alt_h != 0 else height

                    width = s1
                    height = s2

                wpg_unk3 = reader.read_int_array(12, 1)

                printd(f'\tw={width}, h={height}, altw={alt_w}, alth={alt_h}, unk2={wpg_unk2}, bpp={bpp}, bpp_dupe={bpp_dupe}, wpg_unk3={wpg_unk3}, tell={reader.tell()}')
                assert bpp == bpp_dupe

                all_shapes[i] = (width, height)
                all_bpps[i] = bpp
                all_alt_w[i] = alt_w
                all_alt_h[i] = alt_h
                all_wpg_unk3[i] = wpg_unk3

                filler += (width * height)+512
                # if bpp == 16:
                #     filler += (width * height)+1001
                # else:
                #     filler += (width * height)+246
                # check = reader.tell()
                # wpg_unk2 = reader.read_int_array(14*2, size_bytes=1)
                # reader.seek(check)

                # wpg_unk2_p2 = reader.read_int_array(14)

                # for att_bpp in wpg_unk2_p2:
                #     if att_bpp == 8 or att_bpp == 16:
                #         all_bpps[offset] = att_bpp
                #         break

                # all_extra_ints[i] = wpg_unk2_p2
                # # wpg_unk2 = reader.read_byte_array(28)
                # printd(f'tell={reader.tell()}, offset={offset}, im={i}, w={width}, h={height}')
                # printd('\t[1b]', [f'{h:03}' for h in wpg_unk2])
                # printd('\t[2b]', [f'{h:03}' for h in wpg_unk2_p2])
                # printd('\t[1b_x]', [f'0x{h:02X}' for h in wpg_unk2])
                # printd('\t[2b_x]', [f'0x{h:02X}' for h in wpg_unk2_p2])
                # printd('\n')

            filler += 0

            if file_idx+1 < len(offsets):
                next_off = offsets[file_idx+1]
                dist_to_next = next_off - filler
                missing_dist = next_off - (reader.tell()+filler)
                printd(f'\tData starting at {reader.tell()} wants to end at {next_off}, with filler={filler} we get to {reader.tell()+filler} missing {missing_dist}')
                if missing_dist > 0:
                    reader.seek(reader.tell()+missing_dist)
            else:
                next_off = reader.total_bytes()
                dist_to_next = next_off - filler
                missing_dist = next_off - (reader.tell()+filler)
                printd(f'\tData starting at {reader.tell()} wants to end at {next_off}, with filler={filler} we get to {reader.tell()+filler} missing {missing_dist}')
                start_m = reader.tell()
                if missing_dist > 0:
                    reader.seek(reader.tell()+missing_dist)
            # reader.seek(reader.tell()+44)
            # reader.seek(reader.tell()-55)

            # if DEBUG:
            #     ncols = min(wpg_n_images, 3)
            #     nrows = max(1, round(wpg_n_images / ncols))
            #     fig, ax = plt.subplots(nrows, ncols, figsize=(15, 10))
            #     fig.suptitle(f'File {file_idx+1}/{count} at offset {offset}')
            #     print(nrows, ncols)

            # data_start_offset = reader.tell()

            # next_bpp = []
            # for key in all_bpps:
            #     next_bpp.append(all_bpps[key])
            # next_bpp = np.unique(next_bpp)
            # assert len(next_bpp)==1

            # next_bpp = next_bpp[0]
            # if next_bpp == 16:
            #     reader.seek(reader.tell()+32)
            # elif next_bpp == 8:
            #     reader.seek(reader.tell()-55)

            for i in range(wpg_n_images):
                bpp_mode = all_bpps[i]
                shape = all_shapes[i]

                # if bpp_mode == 16:
                #     reader.seek(reader.tell()+128)
                # elif bpp_mode == 8:
                #     reader.seek(reader.tell()+100)
                # else:
                #     printd('\tInvalid bpp before want')
                # if bpp_mode == 16:
                #     want = (shape[0]*shape[1])+1024
                # else:
                #     want = (shape[0]*shape[1])+256
                # bpp_mode = all_extra_ints[i][6]

                want = shape[0]*shape[1]
                printd(f'\tidx={i}, w={shape[0]}, h={shape[1]}, tell={reader.tell()}/{reader.total_bytes()}, want={want}, bpp={bpp_mode}')
                # if bpp_mode == 8:
                #     s1 = min(all_alt_w[i], shape[0]) if all_alt_w[i] != 0 else shape[0]
                #     s2 = min(all_alt_h[i], shape[1]) if all_alt_h[i] != 0 else shape[1]
                #     shape = (s1, s2)
                #     want = shape[0]*shape[1]
                #     printd(f'\tUsing shape {shape} with new want {want}')
                #     # want += 1024//all_alt_w[i]
                # elif bpp_mode == 16:
                #     # want += 1024//all_alt_w[i]
                #     want += 0
                # else:
                #     printd('\tInvalid bpp')

                checkpt = reader.tell()
                im_data = reader.read(want+512)
                if not im_data:
                    # print(f'Ran out of bytes')
                    ran_out_of_bytes.append(op_file)
                    break
                
                after = reader.tell()
                # im = Image.frombytes("P", shape, im_data, 'raw', 'P', 0, -1)
                im = Image.frombytes("P", shape, im_data, 'raw', 'P', 0, -1)

                reader.seek(after-1024)
                arr = reader.read_int_array(1024, size_bytes=1)
                printd(arr)
                reader.seek(after)
                # pal_mode = 'RGB'
                # if pal_mode == 'RGB':
                # else:
                #     arr = reader.read_int_array(1024, size_bytes=1)
                im.putpalette(arr, 'RGBA')
                im = im.convert('RGB')
                im.save(f'figures2/{op_file.stem}_{i}.png')
                # print(np.array(im).shape)
                # np.save('test.npy', np.array(im))
            
                if DEBUG:
                    # if type(ax) != np.ndarray:
                    #     curr = ax
                    # elif len(ax.shape) == 1:
                    #     curr = ax[i]
                    # else:
                    #     r = i // ncols
                    #     c = i % ncols
                    #     curr = ax[r][c]
                    fig, curr = plt.subplots(figsize=(35, 35))
                    curr.set_xticklabels([])
                    curr.set_yticklabels([])
                    curr.imshow(im)
                # curr.set_title(f'i={i+1}/{wpg_n_images}')
                # plt.tight_layout()
                # plt.savefig(f'figures/{i}.png')
            printd(f'\tFinished offset at {reader.tell()} of {reader.total_bytes()}')
        # print(f'Tell finished at {reader.tell()}/{reader.total_bytes()} so {reader.total_bytes()-reader.tell()} bytes remain')
        # print(all_shapes)
        # print(all_n_images)
        all_remains.append(reader.total_bytes()-reader.tell())

        file.close()
    except Exception as ex:
        failed_files.append(op_file)
        if DEBUG:
            ex.with_traceback()
        continue

print('Remains', np.unique(all_remains))
if len(failed_files) > 0:
    print('Failed', len(failed_files), failed_files)
if len(ran_out_of_bytes) > 0:
    print('Ran out of bytes', len(ran_out_of_bytes), ran_out_of_bytes)

# Remains [     0     76    140    172    204 165684]
# Failed 142
# Ran out of bytes 33

  1%|          | 2/182 [00:00<00:13, 13.15it/s]

[3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 3, 4, 5, 128, 4, 9, 11, 128, 4, 11, 14, 128, 5, 14, 18, 128, 7, 17, 22, 128, 7, 20, 25, 128, 8, 20, 27, 128, 9, 22, 30, 128, 15, 31, 42, 128, 12, 34, 41, 128, 16, 33, 44, 128, 16, 35, 47, 128, 14, 38, 46, 128, 17, 36, 49, 128, 7, 42, 57, 128, 18, 37, 50, 128, 7, 24, 27, 128, 8, 26, 31, 128, 10, 24, 33, 128, 12, 26, 35, 128, 10, 29, 36, 128, 13, 28, 38, 128, 14, 30, 41, 128, 6, 34, 45, 128, 19, 38, 52, 128, 15, 41, 51, 128, 19, 39, 53, 128, 16, 44, 53, 128, 20, 41, 55, 128, 19, 46, 56, 128, 23, 43, 58, 128, 8, 51, 68, 128, 19, 48, 59, 128, 20, 49, 58, 128, 24, 46, 63, 128, 21, 50, 63, 128, 25, 48, 66, 128, 9, 56, 76, 128, 22, 52, 66, 128, 26, 50, 70, 128, 25, 57, 74, 128, 7, 64, 92, 128, 29, 56, 77, 128, 33, 54, 75, 128, 11, 64, 91, 128, 27, 59, 77, 128, 8, 66, 95, 128, 30, 58, 80, 128, 29, 49, 67, 128, 22, 55, 69, 128, 27, 52, 72, 128, 8, 60, 85, 128, 24, 56, 71, 128, 32, 51, 71, 128, 7, 62, 88, 128, 28, 54, 74, 128, 7, 67, 95, 128, 11, 67, 93, 128,

  2%|▏         | 4/182 [00:00<00:13, 13.41it/s]

[3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 3, 4, 5, 128, 4, 9, 11, 128, 4, 11, 14, 128, 5, 14, 18, 128, 7, 17, 22, 128, 7, 20, 25, 128, 8, 20, 27, 128, 9, 22, 30, 128, 15, 31, 42, 128, 12, 34, 41, 128, 16, 33, 44, 128, 16, 35, 47, 128, 14, 38, 46, 128, 17, 36, 49, 128, 7, 42, 57, 128, 18, 37, 50, 128, 7, 24, 27, 128, 8, 26, 31, 128, 10, 24, 33, 128, 12, 26, 35, 128, 10, 29, 36, 128, 13, 28, 38, 128, 14, 30, 41, 128, 6, 34, 45, 128, 19, 38, 52, 128, 15, 41, 51, 128, 19, 39, 53, 128, 16, 44, 53, 128, 20, 41, 55, 128, 19, 46, 56, 128, 23, 43, 58, 128, 8, 51, 68, 128, 19, 48, 59, 128, 20, 49, 58, 128, 24, 46, 63, 128, 21, 50, 63, 128, 25, 48, 66, 128, 9, 56, 76, 128, 22, 52, 66, 128, 26, 50, 70, 128, 25, 57, 74, 128, 7, 64, 92, 128, 29, 56, 77, 128, 33, 54, 75, 128, 11, 64, 91, 128, 27, 59, 77, 128, 8, 66, 95, 128, 30, 58, 80, 128, 29, 49, 67, 128, 22, 55, 69, 128, 27, 52, 72, 128, 8, 60, 85, 128, 24, 56, 71, 128, 32, 51, 71, 128, 7, 62, 88, 128, 28, 54, 74, 128, 7, 67, 95, 128, 11, 67, 93, 128,

  4%|▍         | 8/182 [00:00<00:12, 13.95it/s]

...

[0, 0, 0, 0, 29, 0, 0, 0, 0, 0, 29, 54, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 16, 0, 0, 0, 23, 0, 0, 0, 32, 0, 0, 0, 40, 0, 0, 0, 48, 0, 0, 0, 61, 6, 6, 7, 128, 3, 7, 16, 128, 15, 4, 4, 128, 3, 9, 30, 128, 12, 13, 13, 128, 33, 4, 3, 128, 9, 13, 22, 128, 12, 17, 27, 128, 0, 0, 0, 71, 0, 0, 0, 80, 0, 0, 0, 89, 0, 0, 0, 98, 1, 1, 1, 107, 1, 1, 1, 116, 1, 1, 1, 128, 5, 5, 5, 122, 15, 19, 17, 128, 9, 20, 43, 128, 15, 21, 32, 128, 47, 9, 10, 128, 21, 22, 23, 128, 24, 22, 22, 0, 15, 28, 20, 128, 75, 1, 1, 128, 17, 24, 38, 128, 32, 20, 36, 128, 28, 27, 25, 127, 19, 28, 49, 128, 24, 30, 35, 128, 19, 36, 25, 128, 63, 17, 22, 128, 23, 32, 52, 128, 40, 39, 38, 128, 119, 5, 4, 128, 78, 21, 35, 128, 53, 32, 60, 128, 32, 43, 62, 128, 27, 51, 35, 128, 48, 40, 40, 0, 37, 45, 51, 128, 24, 34, 54, 128, 86, 11, 11, 128, 44, 26, 47, 128, 25, 35, 57, 128, 36, 34, 37, 128, 22, 45, 31, 128, 27, 38, 58, 128, 30, 39, 45, 128, 28, 50, 54, 128, 36, 48, 63, 128, 54, 38, 65, 128, 49, 46, 46, 128, 27, 60, 40, 128, 119, 1

 99%|█████████▉| 180/182 [00:04<00:00, 21.12it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 182/182 [00:04<00:00, 38.49it/s]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 6, 7, 2, 6, 2, 6, 4, 6, 7, 7, 6, 3, 6, 3, 6, 4, 6, 4, 6, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 4, 10, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 8, 13, 8, 8, 6, 8, 8, 8, 8, 11, 8, 8, 8, 11, 11, 13, 16, 13, 13, 16, 16, 14, 16, 16, 16, 14, 11, 14, 16, 19, 19, 16, 19, 19, 19, 19, 23, 27, 19, 27, 27, 27, 19, 27, 27, 27, 27, 19, 27, 27, 19, 19, 19, 12, 19, 12, 19, 19, 14, 14, 14, 12, 12, 12, 12, 12, 12, 12, 9, 7, 9, 7, 7, 7, 3, 3, 3, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 3, 3, 7, 7, 7, 7, 7, 7, 12, 7, 7, 7, 7, 7, 7, 12, 12, 7, 12, 7, 12, 7, 12, 7, 12, 7, 12, 12, 12, 7, 7, 12, 12, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 12, 7, 12, 7, 7, 7, 3, 3, 3, 7, 7, 7, 7, 7, 3, 3, 3, 1, 3, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,